In [1]:
from systemml import MLContext, dml, jvm_stdout
ml = MLContext(sc)
ml.version()

u'1.0.0-SNAPSHOT'

In [3]:
script="""
source("nn/layers/cross_entropy_loss.dml") as cross_entropy_loss
source("nn/layers/l2_loss.dml") as l2_loss

source("nn/layers/lstm.dml") as lstm
source("nn/layers/sigmoid.dml") as sigmoid

source("nn/optim/sgd_nesterov.dml") as sgd_nesterov
source("nn/optim/rmsprop.dml") as rmsprop

X = seq(0, 999, 1) * 3.14 / 100 #0 to 10 pi radians

X = sin(X)
#print(toString(X))

in_TS = 75 #in_TS
out_TS = 25 #out_TS
N = nrow(X) / (in_TS + out_TS)
M = out_TS

idx_mat = outer(seq(0,N-1,1), t(seq(0,in_TS+out_TS-1,1)), "+") + 1
#print(toString(idx_mat))

idx_col = matrix(idx_mat, rows=nrow(idx_mat)*ncol(idx_mat), cols=1)
rordrd_X = table(seq(1, nrow(idx_col), 1), idx_col, nrow(idx_col), nrow(idx_col)) %*% X

X = matrix(rordrd_X, rows=nrow(idx_mat), cols=ncol(idx_mat))
#print(toString(X))

Y = X[,in_TS+1:in_TS+out_TS]
X = X[,1:in_TS]

max_iterations = 2000
iter = 0
learningRate = 0.01
decayRate = 0.95

[W, b, out0, c0] = lstm::init(N,1,M)
rmspropCache = rmsprop::init(W)

while( iter < max_iterations ){
    [a1, c, c_out, c_c, c_ifog] = lstm::forward(X, W, b, in_TS, 1, FALSE, out0, c0)
    loss = l2_loss::forward(a1, Y)

    if(iter %% 100 == 0) print("iter=" + iter + " loss=" + loss)

    loss_grad = l2_loss::backward(a1, Y)
    [dX, dW, db, dout0, dc0] = lstm::backward(loss_grad, c0, X, W, b, in_TS, 1, FALSE, out0, c0, c_out, c_c, c_ifog)

    [W, rmspropCache] = rmsprop::update(W, dW, learningRate, decayRate, 1e-6, rmspropCache)

    iter = iter + 1
}

[a1, c, c_out, c_c, c_ifog] = lstm::forward(X, W, b, in_TS, 1, FALSE, out0, c0)
print(toString(cbind(a1, Y)))

"""

In [ ]:
prog = dml(script).output("a1").output("X").output("Y")
#with jvm_stdout(True):
result = ml.execute(prog)

In [ ]:
X = result.get("X").toNumPy()
a1 = result.get("a1").toNumPy()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
np.shape(a1)

In [ ]:
plt.plot(np.transpose(a1))
plt.ylabel('some numbers')
plt.show()

In [ ]:
plt.plot(np.transpose(X))
plt.ylabel('some numbers')
plt.show()

In [98]:
np.transpose(a1)

array([[  7.83449955e-01,   7.92045525e-01,   8.00203088e-01,
          8.07955338e-01,   8.15327076e-01,   8.22337003e-01,
          8.28999197e-01,   8.35324242e-01,   8.41320086e-01,
          8.46992683e-01],
       [  7.88150796e-01,   7.96516557e-01,   8.04375398e-01,
          8.11777655e-01,   8.18762405e-01,   8.25360048e-01,
          8.31594360e-01,   8.37484022e-01,   8.43043746e-01,
          8.48285110e-01],
       [  8.21334251e-01,   8.29588342e-01,   8.37267259e-01,
          8.44438144e-01,   8.51152627e-01,   8.57450660e-01,
          8.63363471e-01,   8.68915731e-01,   8.74127135e-01,
          8.79013560e-01],
       [ -7.86459177e-10,  -7.92806620e-10,  -8.00831516e-10,
         -8.10121717e-10,  -8.20375686e-10,  -8.31364551e-10,
         -8.42907569e-10,  -8.54856103e-10,  -8.67082842e-10,
         -8.79474387e-10],
       [  8.51288578e-01,   8.59023846e-01,   8.66271809e-01,
          8.73074609e-01,   8.79464826e-01,   8.85468035e-01,
          8.91104773e-01

In [ ]:
complete = np.vstack((np.transpose(X),np.transpose(a1)))

In [ ]:
plt.plot(complete)
plt.ylabel('some numbers')
plt.show()